In [1]:
%pip install torch numpy ase torch_geometric scikit-learn pandas rdkit

  Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (114 kB)
  Using cached ase-3.22.1-py3-none-any.whl.metadata (3.1 kB)
  Using cached torch_geometric-2.5.2-py3-none-any.whl.metadata (64 kB)
  Using cached filelock-3.13.4-py3-none-any.whl.metadata (2.8 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.8 MB/s eta 0:00:00
  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached joblib-1.4.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.4.0-py3-none-any.whl.metadata (13 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)


In [13]:
from contextlib import contextmanager
import sys
import os
import torch
from ase import Atoms
from ase.data import chemical_symbols
from ase.calculators.morse import MorsePotential
from ase.optimize import QuasiNewton
import numpy as np

In [3]:
print(os.getcwd())
os.path.exists('../data')

/Users/varunhegde/Documents/Georgia Tech/Spring 2024/MLC/MLC_Final_Project


True

In [5]:
# Loading data
data_dir = os.path.join('..', 'data')
databio_path = os.path.join(data_dir, 'datalist_bio.pt')
datatox_path = os.path.join(data_dir, 'datalist_tox.pt')

datalist_bio = torch.load(databio_path)
datalist_tox = torch.load(datatox_path)

# test that it works
print("Biodegradability Data:")
for data in datalist_bio:
    print(data)
    break

print("Toxicity Data:")
for data in datalist_tox:
    print(data)
    break

Biodegradability Data:
Data(x=[9, 100], formula='C8H7Br', name='', positions=[9, 3], edge_index=[2, 9], edge_attr=[9, 4], bio=[1], tox=[0])
Toxicity Data:
Data(x=[34, 100], formula='C27H25ClN6', name='NCGC00178831-03', positions=[34, 3], edge_index=[2, 37], edge_attr=[37, 4], bio=[0], tox=[1])


In [53]:
datapoint = datalist_tox[0]
def get_molecule(datapoint):
    atomic_numbers = datapoint.x.argmax(dim=1).tolist()
    atomic_symbols = [chemical_symbols[number] for number in atomic_numbers]
    # positions = datapoint.positions.numpy()
    positions = np.random.rand(len(atomic_numbers), 3) * 10  # Random positions within a 10x10x10 Å box
    
    molecule = Atoms(numbers=atomic_numbers, positions=positions)
    return molecule

molecule = get_molecule(datapoint)
print(molecule)

Atoms(symbols='SBCB3CB8CB2CB3CB8CB2', pbc=False)


In [20]:
@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [54]:
def get_dft(molecule, De=0.242, re=0.74, alpha=1.5):
    dft_calculator = MorsePotential(De=De, r0=re, alpha=alpha)
    molecule.set_calculator(dft_calculator)
    energy = molecule.get_potential_energy()
    print(energy)
    with suppress_stdout():
        opt = QuasiNewton(molecule)
        opt.run(fmax=0.02)  # Converge forces to less than 0.02 eV/Angstrom
    # Check the optimized bond length and energy
    optimized_energy = molecule.get_potential_energy()
    return optimized_energy

energy = get_dft(molecule)
print(f'The DFT calculated energy is: {energy} eV')

36.53020122411837
The DFT calculated energy is: -12.000322025798228 eV
